In [1]:
import gym
import itertools
import os
import random
import sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

if "../" not in sys.path:
  sys.path.append("../")

from lib import plotting
from collections import deque, namedtuple

%matplotlib inline

In [2]:
env = gym.envs.make("Breakout-v0")

[2017-03-02 21:16:57,349] Making new env: Breakout-v0


In [3]:
# Atari actions" 0 (no op), 1 (fire), 2 (left), 3 (right)
VALID_ACTIONS = [0, 1, 2, 3]

In [4]:
print(len(VALID_ACTIONS))

4


In [5]:
class StateProcessor(object):
    """A class to process an atari image for input into the nn"""
    def __init__(self):
        # Build the Tensorflow graph.
        with tf.variable_scope('state_processor'):
            # Input is a 210 x 160 x 3 array describing the screen
            self.input_state = tf.placeholder(shape=[210, 160, 3], dtype=tf.uint8)
            # Transform to grayscale
            self.output = tf.image.rgb_to_grayscale(self.input_state)
            # Crop top and bottom of image to make it square
            self.output = tf.image.crop_to_bounding_box(self.output,
                                                        34, 0, 160, 160)
            # resize image to 84 x 84
            self.output = tf.image.resize_images(self.output, [84, 84],
                                               method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
            self.output = tf.squeeze(self.output)
    
    def process(self, sess, state):
        """
        process an atari image
        
        Args:
            sess: A TensorFlow session object
            state: A [210, 160, 3] Atari RGB state
            
        Returns:
            A processed [84, 84, 1] state representing grayscale values
        """
        return sess.run(self.output, {self.input_state: state})
    

In [6]:
class Estimator(object):
    """Neural Network to estimate Q-Value
    
    This network is used for both the Q-Network and Target Network
    """
    
    def __init__(self, scope='estimator', summaries_dir=None):
        self.scope = scope
        # Write Tensorboard summaries to disk
        self.summary_writer = None
        with tf.variable_scope(scope):
            # Build the graph
            self._build_model()
            if summaries_dir:
                summary_dir = os.path.join(summaries_dir, "summaries_{}".format(scope))
                if not os.path.exists(summary_dir):
                    os.makedirs(summary_dir)
                self.summary_writer = tf.summary.FileWriter(summary_dir)
    
    def _build_model(self):
        """
        Builds the Tensorflow graph
        """
        
        # Placeholders for input
        # Input are 4 RGB frames of shape 160, 160 each
        self.X_pl = tf.placeholder(shape=[None, 84, 84, 4], 
                                   dtype=tf.uint8,
                                   name='X')
        # TD Target value
        self.y_pl = tf.placeholder(shape=[None], dtype=tf.float32, name='y')
        # Integer if of which action was selected
        self.actions_pl = tf.placeholder(shape=[None],
                                         dtype=tf.int32,
                                         name='actions')
        # Variables
        layer1_weights = tf.Variable(tf.truncated_normal(
                [8, 8, 4, 32], stddev=0.1))
        layer1_biases = tf.Variable(tf.zeros([32]))
        
        layer2_weights = tf.Variable(tf.truncated_normal(
                [4, 4, 32, 64], stddev=0.1))
        layer2_biases = tf.Variable(tf.zeros([64]))
        
        layer3_weights = tf.Variable(tf.truncated_normal(
                [3, 3, 64, 64], stddev=0.1))
        layer3_biases = tf.Variable(tf.zeros([64]))
        
        layer4_weights = tf.Variable(tf.truncated_normal(
                [7*7*64, 512], stddev=0.1))
        layer4_biases = tf.Variable(tf.zeros([512]))
        
        layer5_weights = tf.Variable(tf.truncated_normal(
                [512, len(VALID_ACTIONS)], stddev=0.1))
        layer5_biases = tf.Variable(tf.zeros([len(VALID_ACTIONS)]))
        
        
        X = tf.to_float(self.X_pl) / 255.0
        batch_size = tf.shape(self.X_pl)[0]

        # Three convolutional layers
        conv = tf.nn.conv2d(X, layer1_weights, [1, 4, 4, 1],  padding='VALID', name='conv1')
        relu = tf.nn.relu(conv + layer1_biases, name='relu1')
        conv = tf.nn.conv2d(relu, layer2_weights, [1, 2, 2, 1], padding='VALID', name='conv2')
        relu = tf.nn.relu(conv + layer2_biases, name='relu2')
        conv = tf.nn.conv2d(relu, layer3_weights, [1, 1, 1, 1], padding='VALID', name='conv3')
        relu = tf.nn.relu(conv + layer3_biases, name='relu3')
        shape = relu.get_shape().as_list()
        
        reshape = tf.reshape(relu, [-1, shape[1]*shape[2]*shape[3]])
        fc1 = tf.matmul(reshape, layer4_weights) + layer4_biases
        self.predictions = tf.matmul(fc1, layer5_weights) + layer5_biases
        
        # Get the predictions for the chosen actions only
        gather_indices = tf.range(batch_size) * tf.shape(self.predictions)[1]
        self.action_predictions = tf.gather(tf.reshape(self.predictions, [-1]), gather_indices)
        
        # Loss
        self.losses = tf.squared_difference(self.y_pl, self.action_predictions)
        self.loss = tf.reduce_mean(self.losses)
        
        # Optimizer Parameters from original paper
        self.optimizer = tf.train.RMSPropOptimizer(0.00025, 0.99, 0.0, 1e-6)
        self.train_op = self.optimizer.minimize(self.loss, 
                                                global_step=tf.contrib.framework.get_global_step())
        
        # Summaries for Tensorboard
        self.summaries = tf.summary.merge([
                tf.summary.scalar('loss', self.loss),
                tf.summary.histogram('loss_hist', self.losses),
                tf.summary.histogram('q_values_hist', self.predictions),
                tf.summary.scalar('max_q_value', tf.reduce_max(self.predictions))
            ])
        
    def predict(self, sess, s):
        """predictions action values.
        
        Args:
            sess: Tensorflow session
            s: State input of shape [batch_size, 4, 160, 160, 3]
                
        Returns:
            Tensor of shape [batch_size, NUM_VALID_ACTIONS] containing
            the action values.
        """
        A = sess.run(self.predictions, {self.X_pl: s})
        
        return A
        #return sess.run(self.predictions, {self.X_pl: s})
        
    def update(self, sess, s, a, y):
        """
        UPdate the estimator toward the given targets
        
        Args:
            sess: Tensorflow session object
            s: State input of size [batch_size, 4, 160, 160, 3]
            a: Chosen action of size [batch_size]
            y: Targets of shape [batch_size]
            
        Returns:
            The calculated loss of each batch
        """
        feed_dict = {self.X_pl: s, self.y_pl: y, self.actions_pl: a}
        summaries, global_step, _, loss = sess.run(
            [self.summaries, 
             tf.contrib.framework.get_global_step(), 
             self.train_op,
             self.loss], feed_dict=feed_dict)
        if self.summary_writer:
            self.summary_writer.add_summary(summaries, global_step)
        return loss

In [7]:
# For Testing

tf.reset_default_graph()
global_step = tf.Variable(0, name='global_step', trainable=False)

e = Estimator(scope='test')
sp = StateProcessor()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    observation = env.reset()
    
    observation_p = sp.process(sess, observation)
    observation = np.stack([observation_p] * 4, axis=2)
    observations = np.array([observation] * 2)
    
    print(e.predict(sess, observations))
    
    # Test training step
    y = np.array([10.0, 10.0])
    a = np.array([1, 3])
    print(e.update(sess, observations, a, y))

/home/cully/anaconda2/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


[[-6.24132586  1.84667373  0.70204931  3.91748405]
 [-6.24132586  1.84667373  0.70204931  3.91748405]]
263.781


In [8]:
def copy_model_parameters(sess, estimator1, estimator2):
    """
    Copies the model params of one estimator to another
    
    Args:
        sess: Tensorflow sesion instance
        estimator1: Estimator to copy the parameters from
        estimator2: Estimator to copy the parameters to
    """
    e1_params = [t for t in tf.trainable_variables() if t.name.startswith(estimator1.scope)]
    e1_params = sorted(e1_params, key=lambda v: v.name)
    e2_params = [t for t in tf.trainable_variables() if t.name.startswith(estimator2.scope)]
    e2_params = sorted(e2_params, key=lambda v: v.name)
    
    update_ops = []
    for e1_v, e2_v in zip(e1_params, e2_params):
        op = e2_v.assign(e1_v)
        update_ops.append(op)
    sess.run(update_ops)

In [9]:
def make_epsilon_greedy_policy(estimator, nA):
    """
    Creates an epsilon-greedy policy based on a given Q-function approximation
    
    Args:
        esimator: An estimator that returns q values for a given state
        nA: Number of actions in the environment
        
    Returns:
        A function that takes the (sess, observation, epsilon) as an argument
        and returns the probabilites for each action in the form of a 
        numpy array of length nA
    """
    
    def policy_fn(sess, observation, epsilon):
        A = np.ones(nA, dtype=float) * epsilon / nA
       
        q_values = estimator.predict(sess, np.expand_dims(observation, 0))[0]
        
        best_action = np.argmax(q_values)
        A[best_action] += (1.0 - epsilon)
        return A
    return policy_fn

In [10]:
def deep_q_learning(sess, 
                    env, 
                    q_estimator, 
                    target_estimator, 
                    state_processor, 
                    num_episodes, 
                    experiment_dir, 
                    replay_memory_size=500000, 
                    replay_memory_init_size=50000, 
                    update_target_estimator_every=10000, 
                    discount_factor=0.99, 
                    epsilon_start=1.0, 
                    epsilon_end=0.1, 
                    epsilon_decay_steps=500000, 
                    batch_size=32, 
                    record_video_every=50):
    """
    Q-learning algorithim for off-policy TD control using function approximation
    Finds the optimal greedy policy while following epsilon-greedy one
    
    Args:
        sess: Tensorflow Session object
        env: OpenAI environment
        q_estimator: Estimator object used for the q values
        target_estimator: Estimator object used for the targets
        state_processor: A StateProcessor object
        num_episodes: Number of episodes to run for
        experiment_dir: Directory to save Tensorflow summaries in
        replay_memory_size: Size of the replay memory
        replay_memory_init_size: Number of random experiences to sampel when initializing 
          the reply memory.
        update_target_estimator_every: Copy parameters from the Q estimator to the 
          target estimator every N steps
        discount_factor: Lambda time discount factor
        epsilon_start: Chance to sample a random action when taking an action.
          Epsilon is decayed over time and this is the start value
        epsilon_end: The final minimum value of epsilon after decaying is done
        epsilon_decay_steps: Number of steps to decay epsilon over
        batch_size: Size of batches to sample from the replay memory
        record_video_every: Record a video every N episodes

    Returns:
        An EpisodeStats object with two numpy arrays for episode_lengths and episode_rewards.
    
    """
    Transition = namedtuple('Transition', ['state', 'action', 'reward', 'next_state', 'done'])
    
    # The replay memory
    replay_memory = deque(maxlen=replay_memory_size)
    
    # Keeps track of useful stats
    stats = plotting.EpisodeStats(
        episode_lengths=np.zeros(num_episodes),
        episode_rewards=np.zeros(num_episodes))
    
    # Create directories for checkpoints and summaries
    checkpoint_dir = os.path.join(experiment_dir, 'checkpoints')
    checkpoint_path = os.path.join(checkpoint_dir, 'model')
    monitor_path = os.path.join(experiment_dir, 'monitor')
    
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    if not os.path.exists(monitor_path):
        os.makedirs(monitor_path)
        
    saver = tf.train.Saver()
    # Load a previous checkpoint if we find one
    latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
    if latest_checkpoint:
        print("Loading model checkpoint {}...\n".format(latest_checkpoint))
        saver.restore(sess, latest_checkpoint)
    
    # Get the current time step
    total_t = sess.run(tf.contrib.framework.get_global_step())
    
    # The epsilon decay schedule
    epsilons = np.linspace(epsilon_start, epsilon_end, epsilon_decay_steps)
    
    # The policy we're following
    policy = make_epsilon_greedy_policy(
        q_estimator,
        len(VALID_ACTIONS))
    
    # Populate the replay memory with initial experience
    print("Populating replay memory...")
    state = env.reset()
    state = state_processor.process(sess, state)
    state = np.stack([state] * 4, axis=2)
    for i in range(replay_memory_init_size):
        if i % 500 == 0:
            print("Step {}".format(i))
        action_probs = policy(sess, state, epsilons[total_t])
        action = np.random.choice(np.arange(len(action_probs)), p=action_probs)
        next_state, reward, done, _ = env.step(VALID_ACTIONS[action])
        next_state = state_processor.process(sess, next_state)
        next_state = np.append(state[:,:,1:], np.expand_dims(next_state, 2), axis=2)
        replay_memory.append(Transition(state, action, reward, next_state, done))
        if done:
            state = env.reset()
            state = state_processor.process(sess, state)
            state = np.stack([state] * 4, axis=2)
        else:
            state = next_state
    
    # Record videos
    env.monitor.start(monitor_path,
                     resume=True,
                     video_callable=lambda count: count % record_video_every)
    
    for i_episode in xrange(num_episodes):
        # Save the current checkpoint
        saver.save(tf.get_default_session(), checkpoint_path)
        
        # Reset the environment
        state = env.reset()
        state = state_processor.process(sess, state)
        state = np.stack([state]*4, axis=2)
        loss = None
        
        # One step in the environment.
        for t in itertools.count():
            epsilon = epsilons[min(total_t, epsilon_decay_steps-1)]
            
            # Add epsilon to Tensorboard
            episode_summary = tf.Summary()
            episode_summary.value.add(simple_value=epsilon, tag='epsilon')
            q_estimator.summary_writer.add_summary(episode_summary, total_t)
            
            if total_t % update_target_estimator_every == 0:
                print("Copying parameters from q_estimator to target...\n")
                copy_model_parameters(sess, q_estimator, target_estimator)
            
            
            # Print step we're on, useful for debugging
            if t % 100 == 0:
                print("\rStep {} ({}) @ Episode {}/{}, loss: {}".format(
                t, total_t, i_episode+1, num_episodes, loss))
                sys.stdout.flush()
            
            # Take a step in the environment
            action_probs = policy(sess, state, epsilon)
            action = np.random.choice(len(action_probs), p=action_probs)
            next_state, reward, done, _ = env.step(VALID_ACTIONS[action])
            next_state = state_processor.process(sess, next_state)
            next_state = np.append(state[:,:,1:], np.expand_dims(next_state, 2), axis=2)
            
            transition = Transition(state, action, reward, next_state, done)
            # If our replay_memory is full, pop the first element
            replay_memory.append(transition)
            
            stats.episode_lengths[i_episode] = t
            stats.episode_rewards[i_episode] += reward
            
            #Sample minibatch from the replay memory
            samples = random.sample(replay_memory, batch_size)
            states_batch, action_batch, reward_batch, next_state_batch, done_batch = map(
            np.array, zip(*samples))
            
            # Calculate q-values and targets
            q_values_next = target_estimator.predict(sess, next_state_batch)
            targets_batch = reward_batch + np.invert(done_batch).astype(np.float32) * discount_factor * np.amax(q_values_next, axis=1)
            
            # Perform gradient descent update
            states_batch = np.array(states_batch)
            loss = q_estimator.update(sess, states_batch, action_batch, targets_batch)
            
            if done:
                break
            
            state = next_state
            total_t += 1
            
            
        # Add summaries to tensorboard
        episode_summary = tf.Summary()
        episode_summary.value.add(simple_value=stats.episode_rewards[i_episode],
                                     node_name='episode_reward', tag='episode_reward')
        episode_summary.value.add(simple_value=stats.episode_lengths[i_episode],
                                     node_name='episode_length', tag='episode_length')
        q_estimator.summary_writer.add_summary(episode_summary, total_t)
        q_estimator.summary_writer.flush()
            
        yield total_t, plotting.EpisodeStats(
            episode_lengths=stats.episode_lengths[:i_episode+1],
            episode_rewards=stats.episode_rewards[:i_episode+1])
        
    env.monitor.close()
    return


In [13]:
tf.reset_default_graph()

# Where we save our checkpoint and graphs
experiment_dir = os.path.abspath("./experiments/{}".format(env.spec.id))

# Create a global_step_variable
global_step = tf.Variable(0, name='global_step', trainable=False)

# Create estimators
q_estimator = Estimator(scope='q', summaries_dir=experiment_dir)
target_estimator = Estimator(scope='target_q')

state_processor = StateProcessor()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for t, stats in deep_q_learning(sess,
                                   env,
                                   q_estimator=q_estimator,
                                   target_estimator=target_estimator,
                                   state_processor=state_processor,
                                    num_episodes=200,
                                   experiment_dir=experiment_dir,
                                   replay_memory_size=500000,
                                   replay_memory_init_size=50000):
        print("\nEpisode Reward: {}".format(stats.episode_rewards[-1]))

Loading model checkpoint /home/cully/git/reinforcement-learning/DQN/experiments/Breakout-v0/checkpoints/model...

Populating replay memory...
Step 0
Step 500
Step 1000
Step 1500
Step 2000
Step 2500
Step 3000
Step 3500
Step 4000
Step 4500
Step 5000
Step 5500
Step 6000
Step 6500
Step 7000
Step 7500
Step 8000
Step 8500
Step 9000
Step 9500
Step 10000
Step 10500
Step 11000
Step 11500
Step 12000
Step 12500
Step 13000
Step 13500
Step 14000
Step 14500
Step 15000
Step 15500
Step 16000
Step 16500
Step 17000
Step 17500
Step 18000
Step 18500
Step 19000
Step 19500
Step 20000
Step 20500
Step 21000
Step 21500
Step 22000
Step 22500
Step 23000
Step 23500
Step 24000
Step 24500
Step 25000
Step 25500
Step 26000
Step 26500
Step 27000
Step 27500
Step 28000
Step 28500
Step 29000
Step 29500
Step 30000
Step 30500
Step 31000
Step 31500
Step 32000
Step 32500
Step 33000
Step 33500
Step 34000
Step 34500
Step 35000
Step 35500
Step 36000
Step 36500
Step 37000
Step 37500
Step 38000
Step 38500
Step 39000
Step 39500
St

[2017-03-02 23:33:55,186] Starting new video recorder writing to /home/cully/git/reinforcement-learning/DQN/experiments/Breakout-v0/monitor/openaigym.video.2.4056.video000001.mp4



Episode Reward: 2.0
Step 0 (80136) @ Episode 2/200, loss: None
Step 100 (80236) @ Episode 2/200, loss: 0.0593302287161


[2017-03-02 23:34:05,995] Starting new video recorder writing to /home/cully/git/reinforcement-learning/DQN/experiments/Breakout-v0/monitor/openaigym.video.2.4056.video000002.mp4



Episode Reward: 0.0
Step 0 (80312) @ Episode 3/200, loss: None
Step 100 (80412) @ Episode 3/200, loss: 0.0174886547029
Step 200 (80512) @ Episode 3/200, loss: 0.0129819680005


...


Episode Reward: 1.0
Step 0 (130119) @ Episode 199/200, loss: None
Step 100 (130219) @ Episode 199/200, loss: 0.0441041067243

Episode Reward: 0.0


[2017-03-03 00:28:37,826] Starting new video recorder writing to /home/cully/git/reinforcement-learning/DQN/experiments/Breakout-v0/monitor/openaigym.video.2.4056.video000199.mp4


Step 0 (130291) @ Episode 200/200, loss: None
Step 100 (130391) @ Episode 200/200, loss: 0.0409513115883
Step 200 (130491) @ Episode 200/200, loss: 0.0338000915945


[2017-03-03 00:28:55,020] Finished writing results. You can upload them to the scoreboard via gym.upload('/home/cully/git/reinforcement-learning/DQN/experiments/Breakout-v0/monitor')



Episode Reward: 1.0


In [16]:
stats.

array([ 1.,  0.,  1.,  3.,  0.,  0.,  0.,  1.,  1.,  1.])

In [104]:
state = env.reset()

In [121]:
Transition = namedtuple('Transition', ['state', 'action', 'reward', 'next_state', 'done'])
state = env.reset()
print state.shape
with tf.Session() as sess:
    state = sp.process(sess, state)
    #print(state)
    print(state.shape)
    state = np.stack([state]*4, axis=2)
    print(state.shape)
    action = np.random.randint(4)
    next_state, reward, done, _ = env.step(action)
    print(next_state.shape)
    next_state = sp.process(sess, next_state)
    print(next_state.shape)
    print(state[:,:,1:].shape)
    next_state = np.append(state[:,:,1:], np.expand_dims(next_state, 2), axis=2)
    print(next_state.shape)
    trans = Transition(state, action, reward, next_state, done)
    print(trans)

(210, 160, 3)
(84, 84)
(84, 84, 4)
(210, 160, 3)
(84, 84)
(84, 84, 3)
(84, 84, 4)
Transition(state=array([[[142, 142, 142, 142],
        [142, 142, 142, 142],
        [142, 142, 142, 142],
        ..., 
        [142, 142, 142, 142],
        [142, 142, 142, 142],
        [142, 142, 142, 142]],

       [[142, 142, 142, 142],
        [142, 142, 142, 142],
        [142, 142, 142, 142],
        ..., 
        [142, 142, 142, 142],
        [142, 142, 142, 142],
        [142, 142, 142, 142]],

       [[142, 142, 142, 142],
        [142, 142, 142, 142],
        [142, 142, 142, 142],
        ..., 
        [142, 142, 142, 142],
        [142, 142, 142, 142],
        [142, 142, 142, 142]],

       ..., 
       [[142, 142, 142, 142],
        [142, 142, 142, 142],
        [142, 142, 142, 142],
        ..., 
        [142, 142, 142, 142],
        [142, 142, 142, 142],
        [142, 142, 142, 142]],

       [[127, 127, 127, 127],
        [127, 127, 127, 127],
        [127, 127, 127, 127],
        ..., 
